In [2]:
import pandas as pd

In [5]:
from textblob import TextBlob
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [42]:
import re

In [6]:
!pip install spacy-lookups-data
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.2.0-py2.py3-none-any.whl size=29164786 sha256=861f5bf363fd6c6b7c134f5fb97f8dd2678b80c55b363bb6b3cb19261846308d
  Stored in directory: C:\Users\angelica.jara\AppData\Local\pip\Cache\wheels\79\a4\b8\6085d282396938b29675292697e72871b145990d0079ceadc1
Successfully built spacy-lookups-data
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp37-none-any.whl size=12011744 sha256=d69d73f538df7034a29d54fe64502704d95a799f7c1f3754704482f2211480e6
  Stored in directory: C:\Users\ANGELI~1.JAR\AppData\Local\Temp\pip-ephem-wheel-cache-vga0jv23\wheels\6a\47\fb\6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172937 sha256=0ec5756bf8571059eeb5d89868137d90166dc921ef03aa387aaad70b4790d847
  Sto

In [14]:
import spacy

In [28]:
import es_core_news_sm as nlp

In [29]:
import seaborn as sns

In [24]:
dataset7s = pd.read_csv('Seven_Sins_Restobar.tsv', delimiter = '\t', quoting = 3, encoding = 'unicode_escape')

# Descriptive Analysis

In [25]:
dataset7s.head()

Review  Liked
0  "I also had to taste my Mom's multi-grain pump...      1
1          "I'd rather eat airline food, seriously."      0
2      Cant say enough good things about this place.      1
3                       The ambiance was incredible.      1
4          The waitress and manager are so friendly.      1

In [26]:
dataset7s.shape

(500, 2)

In [32]:
liked=dataset7s.Liked

In [34]:
liked.value_counts(normalize=True)

1    0.5
0    0.5
Name: Liked, dtype: float64

In [35]:
dataset7s.describe()

Liked
count  500.000000
mean     0.500000
std      0.500501
min      0.000000
25%      0.000000
50%      0.500000
75%      1.000000
max      1.000000

In [40]:

sns.countplot(x=liked, data=dataset7s)

In [41]:
dataset7s.isnull().sum()

Review    0
Liked     0
dtype: int64

Considering we are working with dummy variables the only descriptive statistical analysis that is useful is that we have 50% of likes and 50% of dislikes. 

In [48]:
dataset7s.columns

Index(['Review', 'Liked'], dtype='object')

# Main Objective

Analyzing the case and the data we can say that our object of study is the final output of the reviews, meaning if they are good or bad. But also we have to consider analyzing what was the drove to that final review in order to obtain better results in the future.
On the other hand we already observed that we have 50% of good and bad reviews which is mixed review in the end, our main objective is to obtain a model that helps improve our reviews in the future to attract new customers.

# Preprocessing and methods

In [43]:
from nltk.corpus import stopwords
from nl tk.stem.porter import PorterStemmer

In [78]:
corpus = [] 

for i in range(0, 500):
    review = re.sub('[^a-zA-Z]', ' ', dataset7s['Review'][i]) #Removes all but letters (and Capitals)
    review = review.lower() #Put all the letters in lowercase
    review = review.split() #Creates a list of different words
    ps = PorterStemmer() #We use Stemming for taking the root of words
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) #For reversing the process (list -> string)
    corpus.append(review) #Generate a list of one string per observation


In [52]:
from sklearn.feature_extraction.text import CountVectorizer #Class that allows tokenization

cv = CountVectorizer(max_features = 1000) #Keepong the 1000 more frequent words

X = cv.fit_transform(corpus).toarray() #Creation of the Sparsematrix

y = dataset7s.iloc[:, 1].values #Defining our response variable


The data was cleaned in order to deliver different analysis through the possible models

#    Model Analysis

In [53]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
!pip install tabulate
from tabulate import tabulate
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23279 sha256=f4bab02b5d7921d398e9274e49cb09ad401d31ed19a05413d0a4b30faec21f3f
  Stored in directory: C:\Users\angelica.jara\AppData\Local\pip\Cache\wheels\9c\9b\f4\eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
Successfully built tabulate


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set

from sklearn.naive_bayes import GaussianNB
classifier_nb = GaussianNB()
classifier_nb.fit(X_train, y_train)

# fit a Logistic Regression Model
classifier_lr = LogisticRegression(solver='lbfgs')
classifier_lr.fit(X_train, y_train)

# Predicting the Test set results
nb_y_pred = classifier_nb.predict(X_test)
lr_y_pred = classifier_lr.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
nb_cm = confusion_matrix(y_test, nb_y_pred)
print(nb_cm)
lr_cm = confusion_matrix(y_test, lr_y_pred)
print(lr_cm)


[[27 23]
 [ 6 44]]
[[35 15]
 [15 35]]


In [61]:
# predict probabilities
nb_probs = classifier_nb.predict_proba(X_test)
lr_probs = classifier_lr.predict_proba(X_test)

# keep probabilities for the positive outcome only
ns_probs = [0 for _ in range(len(y_test))]
nb_probs = nb_probs[:, 1]
lr_probs = lr_probs[:, 1]

# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
nb_auc = roc_auc_score(y_test, nb_probs)
lr_auc = roc_auc_score(y_test, lr_probs)

# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Naive Bayes: ROC AUC=%.3f' % (nb_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, ns_thresholds  = roc_curve(y_test, ns_probs)
nb_fpr, nb_tpr, nb_thresholds  = roc_curve(y_test, nb_probs)
lr_fpr, lr_tpr, lr_thresholds  = roc_curve(y_test, lr_probs)

#get roc curve metrics
from IPython.display import display

ns_df = pd.DataFrame(data=[])
ns_df['threshold'] = ns_thresholds
ns_df['tpr'] = ns_tpr
ns_df['fpr'] = ns_fpr
display(ns_df)

nb_df = pd.DataFrame(data=[])
nb_df['threshold'] = nb_thresholds
nb_df['tpr'] = nb_tpr
nb_df['fpr'] = nb_fpr
display(nb_df)

lr_df = pd.DataFrame(data=[])
lr_df['threshold'] = lr_thresholds
lr_df['tpr'] = lr_tpr
lr_df['fpr'] = lr_fpr
display(lr_df)

# plot the roc curve for the model

fig, ax1 = pyplot.subplots()

ax1.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
ax1.plot(nb_fpr, nb_tpr, marker='.', label='Naive Bayes')
ax1.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')

# axis labels
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')

# show the legend
pyplot.legend()





No Skill: ROC AUC=0.500
Naive Bayes: ROC AUC=0.710
Logistic: ROC AUC=0.768


threshold  tpr  fpr
0          1  0.0  0.0
1          0  1.0  1.0

threshold   tpr   fpr
0        2.0  0.00  0.00
1        1.0  0.88  0.46
2        0.0  1.00  1.00

threshold   tpr   fpr
0    1.999911  0.00  0.00
1    0.999911  0.02  0.00
2    0.963908  0.10  0.00
3    0.952055  0.10  0.02
4    0.808690  0.34  0.02
5    0.794672  0.34  0.06
6    0.771599  0.40  0.06
7    0.751255  0.40  0.10
8    0.735760  0.42  0.10
9    0.733391  0.42  0.12
10   0.702609  0.52  0.12
11   0.697616  0.52  0.14
12   0.681008  0.54  0.14
13   0.676496  0.54  0.16
14   0.674784  0.56  0.16
15   0.663734  0.56  0.18
16   0.640720  0.58  0.18
17   0.634032  0.58  0.20
18   0.586230  0.64  0.20
19   0.567556  0.64  0.22
20   0.554834  0.66  0.22
21   0.544225  0.66  0.26
22   0.526270  0.70  0.26
23   0.485879  0.70  0.34
24   0.470072  0.72  0.34
25   0.466426  0.72  0.36
26   0.452494  0.76  0.36
27   0.428239  0.76  0.42
28   0.392767  0.84  0.42
29   0.381911  0.84  0.46
30   0.381492  0.86  0.46
31   0.350426  0.86  0.52
32   0.348744  0.88  0.52
33   0.320811  0.88  0.62
34   0.315424  0.90  0.62
35   0.266787  0.90  0.74
36   0.261844  0.92  0.74
37   0.253786  0.92  0.80
38   0.238886  0.94  0.80
39   0.211520  0.94  0.82
40   0.211244  0.96  0.82
41   0.155739  0.96  0.94
42   0.137476  0.98  0.94
43   0.088363  0.98  1.00
44   0.049517  1.00  1.00

In [63]:
# define a function that accepts a threshold and prints sensitivity and specificity
def evaluate_threshold_lr(threshold):
    print('LR Sensitivity:', lr_tpr[lr_thresholds > threshold][-1])
    print('LR Specificity:', 1 - lr_fpr[lr_thresholds > threshold][-1])
    
def evaluate_threshold_nb(threshold):
    print('NB Sensitivity:', nb_tpr[nb_thresholds > threshold][-1])
    print('NB Specificity:', 1 - nb_fpr[nb_thresholds > threshold][-1])

evaluate_threshold_lr(5.654338e-41)
evaluate_threshold_nb(5.654338e-41)

evaluate_threshold_lr(0.357685)
evaluate_threshold_nb(0.357685)

evaluate_threshold_lr(0.5)
evaluate_threshold_nb(0.5)

evaluate_threshold_lr(0.75)
evaluate_threshold_nb(0.75)

LR Sensitivity: 1.0
LR Specificity: 0.0
NB Sensitivity: 0.88
NB Specificity: 0.54
LR Sensitivity: 0.86
LR Specificity: 0.54
NB Sensitivity: 0.88
NB Specificity: 0.54
LR Sensitivity: 0.7
LR Specificity: 0.74
NB Sensitivity: 0.88
NB Specificity: 0.54
LR Sensitivity: 0.4
LR Specificity: 0.9
NB Sensitivity: 0.88
NB Specificity: 0.54


# Report of the Analysis

 Observing the analysis of the models that the one that performs the best is the Logistic Model with a ROC AUC = 0.768, meaning that 76.8% of the predictions were correct by the model.
 To get to this result the process was order the data to put it in the models, then we chose a trainset of 80% and a test set of 20% for the models and we analyzed the Sensitivity and Specificity of the models.
 
The results are that if we choose the Logistic model we are putting the sensitivity above the specificity but in this case the specificity is too high so we should consider that as the ROC AUC of the Logistic model is below 0.8 we should try to develop a decision tree model(I tried but I couldn't finish feeding the model).

Taking the main objective of improving the good reviews and considering the accuracy of the model even though it's not precised it could be improved by analyzing the words and taking in consideration the information delivered by the reviews analyzing the most used words that can be reviewed below.

To improve the good reviews it's important to combine a quantitative analysis and a qualitative analysis, in this case the quantitative one it's not as good as we would like it would be good to have a threshold of 0.85 and above to secure a good analysis.

# Decision Tree Analysis

In [197]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [258]:
from sklearn.feature_extraction.text import CountVectorizer #Class that allows tokenization

cv = CountVectorizer(max_features = 1000) #Keepong the 1000 more frequent words

X2 = cv.fit_transform(corpus).toarray() #Creation of the Sparsematrix

y2 = dataset7s.iloc[:, 1].values #Defining our response variable

In [259]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into train and test 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 0) 

      
# Function to perform training with giniIndex. 
def train_using_gini(X2_train, X2_test, y2_train): 
  
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 0,max_depth=3, min_samples_leaf=5) 
  
    # Performing training 
    clf_gini.fit(X2_train, y2_train) 
    return clf_gini 
      
# Function to perform training with entropy. 
def train_using_entropy(X2_train, X2_test, y2_train): 
  
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 0, 
            max_depth = 3, min_samples_leaf = 5) 
  
    # Performing training 
    clf_entropy.fit(X2_train, y2_train) 
    return clf_entropy 
  

In [260]:
# Function to make predictions 
def prediction(X2_test, clf_object): 
  
    # Predicton on test with giniIndex 
    y2_pred = clf_object.predict(X2_test) 
    print("Predicted values:") 
    print(y2_pred) 
    return y2_pred 
      


In [261]:
# Function to calculate accuracy 
def cal_accuracy(y2_test, y2_pred): 
      
    print("Confusion Matrix: ", confusion_matrix(y2_test, y2_pred)) 
      
    print ("Accuracy : ", accuracy_score(y2_test,y2_pred)*100) 
      
    print("Report : ", classification_report(y2_test, y2_pred)) 

In [250]:
def main(): 
      
    # Building Phase 
    data = importdata(dataset7s) 
    X2, y2, X2_train, X2_test, y2_train, y2_test = splitdataset(dataset7s) 
    clf_gini = train_using_gini(X2_train, X2_test, y2_train) 
    clf_entropy = tarin_using_entropy(X2_train, X_2test, y2_train) 
      
    # Operational Phase 
    print("Results Using Gini Index:") 
      
    # Prediction using gini 
    y_pred_gini = prediction(X2_test, clf_gini) 
    cal_accuracy(y2_test, y2_pred_gini) 
      
    print("Results Using Entropy:") 
    # Prediction using entropy 
    y2_pred_entropy = prediction(X2_test, clf_entropy) 
    cal_accuracy(y2_test, y2_pred_entropy) 
      

# Analysis of the words

Below you will find the most common words used in the reviews. Things to consider the waiting time, the customer service, the type of foods and the suggestions should be considered. 
Consider the types of food people like to improve the offer of the restaurant and in consecuence improve the reviews.

In [102]:
from collections import Counter
common=Counter(" ".join(dataset7s["Review"]).split()).most_common(300)

In [103]:
dfcommon=pd.DataFrame(common) 

In [112]:
dfcommon.head()

0    1
0  and  181
1  the  174
2  was  143
3    I  133
4    a  112

In [119]:
dfcommon.head(70)

0    1
0       and  181
1       the  174
2       was  143
3         I  133
4         a  112
..      ...  ...
65     been   11
66  chicken   11
67      eat   10
68   place.   10
69       My   10

[70 rows x 2 columns]

In [81]:
reviews=dataset7s.Review

In [124]:
type(reviews)

pandas.core.series.Series

In [129]:
reviews.apply(str)

0      "I also had to taste my Mom's multi-grain pump...
1              "I'd rather eat airline food, seriously."
2          Cant say enough good things about this place.
3                           The ambiance was incredible.
4              The waitress and manager are so friendly.
                             ...                        
495    "Worse of all, he humiliated his worker right ...
496                      CONCLUSION: Very filling meals.
497    Their daily specials are always a hit with my ...
498    The pancake was also really good and pretty la...
499    "This was my first crawfish experience, and it...
Name: Review, Length: 500, dtype: object

In [133]:
dataset7s['Review'].astype(str)

0      "I also had to taste my Mom's multi-grain pump...
1              "I'd rather eat airline food, seriously."
2          Cant say enough good things about this place.
3                           The ambiance was incredible.
4              The waitress and manager are so friendly.
                             ...                        
495    "Worse of all, he humiliated his worker right ...
496                      CONCLUSION: Very filling meals.
497    Their daily specials are always a hit with my ...
498    The pancake was also really good and pretty la...
499    "This was my first crawfish experience, and it...
Name: Review, Length: 500, dtype: object

In [142]:
reviewsdf = reviews.to_frame()

In [144]:
type(reviewsdf)

pandas.core.frame.DataFrame

In [155]:
reviewsdf.head()

Review
0  "I also had to taste my Mom's multi-grain pump...
1          "I'd rather eat airline food, seriously."
2      Cant say enough good things about this place.
3                       The ambiance was incredible.
4          The waitress and manager are so friendly.

In [160]:
reviewsdfstr= reviewsdf['Review'].to_string()


In [161]:
tokens = [t for t in reviewsdfstr.split()]
 
print (tokens)

['0', '"I', 'also', 'had', 'to', 'taste', 'my', "Mom's", 'multi-grain', 'pump...', '1', '"I\'d', 'rather', 'eat', 'airline', 'food,', 'seriously."', '2', 'Cant', 'say', 'enough', 'good', 'things', 'about', 'this', 'place.', '3', 'The', 'ambiance', 'was', 'incredible.', '4', 'The', 'waitress', 'and', 'manager', 'are', 'so', 'friendly.', '5', 'I', 'would', 'not', 'recommend', 'this', 'place.', '6', 'Overall', 'I', "wasn't", 'very', 'impressed', 'with', 'Noca.', '7', 'My', 'gyro', 'was', 'basically', 'lettuce', 'only.', '8', 'Terrible', 'service!', '9', 'Thoroughly', 'disappointed!', '10', '"I', "don't", 'each', 'much', 'pasta,', 'but', 'I', 'love', 'the', 'homem...', '11', '"Give', 'it', 'a', 'try,', 'you', 'will', 'be', 'happy', 'you', 'did."', '12', 'By', 'far', 'the', 'BEST', 'cheesecurds', 'we', 'have', 'ever', 'had!', '13', 'Reasonably', 'priced', 'also!', '14', 'Everything', 'was', 'perfect', 'the', 'night', 'we', 'were', 'in.', '15', 'The', 'food', 'is', 'very', 'good', 'for', 'yo